### 4. Model Complexity & Practical Usability (모델 복잡도 및 실용성)

이 표는 각 모델이 **얼마나 계산적으로 무거운지**, 그리고 실제 사용할 때 **연산/메모리 비용**이 어느 정도인지 비교합니다.

| Metric | 의미 (Korean 설명) |
|-------|----------------|
| **Params (#)** | 학습 가능한 파라미터 총 개수. 모델 표현력 규모를 반영하나, 너무 크면 과적합 및 메모리 비용 증가 가능. |
| **FLOPs** | 단일 추론(Forward pass) 동안 수행되는 부동소수점 연산 수. 연산 복잡도의 직접적인 척도. |
| **Inference Memory (MB)** | 입력 1개를 추론할 때 GPU 메모리가 어느 정도 사용되는지. |
| **Latency per Inference (s)** | 입력 하나를 처리하는 데 걸리는 시간. 실시간 처리 가능성 및 배치 사이즈 결정에 영향. |

#### 해석 관점
- **ViT** 계열은 일반적으로 **파라미터 수는 크지만 FLOPs 효율이 좋아** 추론 속도는 빠른 편.
- **UNet3D (V-NET)** 는 **입체 convolution 핵심 구조로 인해 메모리 사용량이 크고 추론 시간이 상대적으로 길 수 있음.**
- **Base Model** 은 구조가 단순하므로 일반적으로 가장 가볍지만 성능 한계가 존재.

즉,
> 이 표는 “**정확도 vs 계산비용**” 트레이드오프를 정량적으로 보여주며,  
> 실제 운용 환경에서 어떤 모델을 선택해야 하는지를 결정하는 핵심 기준이 됩니다.


In [1]:
# <<< 이 셀을 노트북 "맨 위"에서 실행하세요 >>>
import os
# TF가 GPU를 전혀 보지 못하도록 비활성화 (CPU 강제)
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import tensorflow as tf
# GPU가 안 보이므로 굳이 메모리 그로스 설정은 불필요

# PyTorch는 별도 환경에서 GPU 사용 (CUDA_VISIBLE_DEVICES가 빈 문자열이면 CPU만 보임)
# -> Torch쪽에서는 다시 원하는 GPU를 지정해서 사용하세요 (SLURM 스크립트 등에서 지정)


2025-11-27 11:43:21.417452: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-27 11:43:32.568488: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [13]:
import math
import time
import pandas as pd
import torch
from torch import nn
from thop import profile

from src.model import VDM, CUNet


# ============================================================
# 1. VDM + CUNet 생성
# ============================================================
def build_vdm_model(spatial_shape=(128, 128, 128), s_cond_ch=2):
    D, H, W = spatial_shape
    score_shape = (1, D, H, W)

    score_model = CUNet(
        shape=score_shape,
        out_channels=1,
        s_conditioning_channels=s_cond_ch,
        v_conditioning_dims=[],
        v_conditioning_type="common_zerolinear",
        v_embedding_dim=64,
        v_augment=False,
        v_embed_no_s_gelu=False,
        t_conditioning=True,
        t_embedding_dim=64,
        init_scale=0.02,
        num_res_blocks=1,
        norm_groups=8,
        mid_attn=False,
        n_attention_heads=4,
        dropout_prob=0.1,
        conv_padding_mode="zeros",
        verbose=0,
    )

    vdm = VDM(
        score_model=score_model,
        noise_schedule="fixed_linear",
        gamma_min=-13.3,
        gamma_max=5.0,
        antithetic_time_sampling=True,
        data_noise=1e-3,
    )
    return vdm


# ============================================================
# 2. Complexity 측정
# ============================================================
def measure_vdm_complexity():
    """
    - FLOPs/MACs: score_model(CUNet) 한 번 forward 기준
    - Memory/Latency: VDM.get_loss 한 번 호출 기준
    """
    device = torch.device("cpu")  # GPU OOM 피하려고 CPU 기준으로 측정

    model = build_vdm_model(spatial_shape=(128, 128, 128), s_cond_ch=2).to(device)
    model.eval()

    B = 1
    zt = torch.randn(B, 1, 128, 128, 128, device=device)
    s_cond = torch.randn(B, 2, 128, 128, 128, device=device)
    y = torch.randn(B, 1, 128, 128, 128, device=device)
    t = torch.zeros(B, device=device)  # dummy t

    # ---------------- FLOPs / MACs (score_model) ----------------
    class ScoreWrapper(nn.Module):
        def __init__(self, score_model, t, s_cond):
            super().__init__()
            self.score_model = score_model
            self.register_buffer("t", t)
            self.register_buffer("s_cond", s_cond)

        def forward(self, x):
            return self.score_model(x, t=self.t, s_conditioning=self.s_cond)

    wrapper = ScoreWrapper(model.score_model, t, s_cond).to(device)
    wrapper.eval()

    macs, params_thop = profile(wrapper, inputs=(zt,), verbose=False)
    flops = macs * 2  # 관례적으로 FLOPs ≈ 2 * MACs

    # ---------------- 파라미터 수 (VDM 전체) ----------------
    params = sum(p.numel() for p in model.parameters())

    # ---------------- 메모리 (GPU 아니면 NaN) ----------------
    if device.type == "cuda":
        torch.cuda.reset_peak_memory_stats(device)

    with torch.no_grad():
        _ = model.get_loss(x=y, s_conditioning=s_cond, reduction="mean")

    if device.type == "cuda":
        peak_mem = torch.cuda.max_memory_allocated(device) / (1024 ** 2)
    else:
        peak_mem = float("nan")

    # ---------------- Latency (get_loss) ----------------
    runs = 5
    with torch.no_grad():
        for _ in range(2):  # warm-up
            _ = model.get_loss(x=y, s_conditioning=s_cond, reduction="mean")

        if device.type == "cuda":
            torch.cuda.synchronize(device)

        start = time.time()
        for _ in range(runs):
            _ = model.get_loss(x=y, s_conditioning=s_cond, reduction="mean")
        if device.type == "cuda":
            torch.cuda.synchronize(device)
        end = time.time()

    latency = (end - start) / runs

    return {
        "Model": "VDM (CUNet backbone)",
        "Params (#)": float(params),
        "FLOPs": float(flops),
        "MACs": float(macs),
        "Inference Memory (MB)": float(peak_mem),
        "Latency per Inference (s)": float(latency),
    }


# ============================================================
# 3. 기존 CSV + VDM 행 추가 + 포맷 통일
# ============================================================
csv_path = "/home/mingyeong/GAL2DM_ASIM_VNET/eval/model_complexity_summary_all_with_cgan.csv"
df = pd.read_csv(csv_path)

vdm_row = measure_vdm_complexity()
df = pd.concat([df, pd.DataFrame([vdm_row])], ignore_index=True)

# ----- 포맷 함수들 -----
def fmt_params(n):
    if isinstance(n, str): return n
    if math.isnan(n): return "NaN"
    if n >= 1e9:  return f"{n/1e9:.3f} B"
    if n >= 1e6:  return f"{n/1e6:.3f} M"
    if n >= 1e3:  return f"{n/1e3:.3f} K"
    return f"{n:.0f}"

def fmt_large_unit(x):
    """FLOPs, MACs 공통 포맷 (M / G / T 단위)."""
    if isinstance(x, str): return x
    if math.isnan(x): return "NaN"
    if x >= 1e12: return f"{x/1e12:.3f}T"
    if x >= 1e9:  return f"{x/1e9:.3f}G"
    if x >= 1e6:  return f"{x/1e6:.3f}M"
    if x >= 1e3:  return f"{x/1e3:.3f}K"
    return f"{x:.0f}"

def fmt_mem(x):
    if isinstance(x, str): return x
    if math.isnan(x): return "NaN"
    return f"{x:,.0f} MB"

def fmt_lat(x):
    if isinstance(x, str): return x
    if math.isnan(x): return "NaN"
    return f"{x*1e3:.2f} ms"

# ----- 각 컬럼에 포맷 적용 -----
if "Params (#)" in df.columns:
    df["Params (#)"] = df["Params (#)"].apply(fmt_params)
if "FLOPs" in df.columns:
    df["FLOPs"] = df["FLOPs"].apply(fmt_large_unit)
if "MACs" in df.columns:
    df["MACs"] = df["MACs"].apply(fmt_large_unit)
if "Inference Memory (MB)" in df.columns:
    df["Inference Memory (MB)"] = df["Inference Memory (MB)"].apply(fmt_mem)
if "Latency per Inference (s)" in df.columns:
    df["Latency per Inference (s)"] = df["Latency per Inference (s)"].apply(fmt_lat)

order = ["Model", "Params (#)", "FLOPs", "MACs",
         "Inference Memory (MB)", "Latency per Inference (s)"]
df = df[[c for c in order if c in df.columns]]

print("\n=== Model Complexity Summary (Including VDM) ===\n")
print(df.to_string(index=False))



=== Model Complexity Summary (Including VDM) ===

                       Model Params (#)    FLOPs     MACs Inference Memory (MB) Latency per Inference (s)
              V-NET (UNet3D)   28.824 M 121.235G  60.617G                937 MB                  30.64 ms
        ViT (3D Transformer)   23.502 M   2.282T   1.141T              2,994 MB                 119.53 ms
cGAN (Pix2PixCC3D-Generator)   27.808 M 953.571G 476.785G              1,100 MB                 211.51 ms
             Base Model (TF)  461.007 M   1.840T      NaN              6,002 MB                 248.64 ms
        VDM (CUNet backbone)   38.993 M   2.777T   1.389T                   NaN                2509.18 ms
